# Prompts & Prompt Engineering

## Install Azure OpenAI .NET SDK & other NuGet packages

In [1]:
#r "nuget: Azure.AI.OpenAI, 1.0.0-beta.15"
#r "nuget: Microsoft.DotNet.Interactive, 1.0.0-beta.24164.1"

Installed Packages Azure.AI.OpenAI, 1.0.0-beta.15 Microsoft.DotNet.Interactive, 1.0.0-beta.24164.1

In [2]:
using System;
using Azure.AI.OpenAI;
using Microsoft.DotNet.Interactive;

## Get Azure OpenAI Service credentials

In [3]:
var endpoint = Environment.GetEnvironmentVariable("AOAI_ENDPOINT");
var key = Environment.GetEnvironmentVariable("AOAI_KEY");
var deploymentId = Environment.GetEnvironmentVariable("AOAI_DEPLOYMENTID");

## Configure OpenAI Client

In [4]:
var client = new OpenAIClient(new Uri(endpoint), new Azure.AzureKeyCredential(key));

## Generic prompts

In [5]:
var genericSystemPrompt = """You are an AI assistant that helps people find information about your business""";

In [6]:
var options = new ChatCompletionsOptions
{
    DeploymentName=deploymentId,
    MaxTokens=400,
    Temperature=1f,
    FrequencyPenalty=0.0f,
    PresencePenalty=0.0f,
    NucleusSamplingFactor = 0.95f // Top P
};

### Initialize chat history

In [8]:
var systemMessage = new ChatRequestSystemMessage(genericSystemPrompt);
options.Messages.Add(systemMessage);

### Start chat

In [9]:
var chatting = true;

In [10]:
Console.WriteLine($"System: {genericSystemPrompt}");
while(chatting)
{
    var userPrompt = await Kernel.GetInputAsync("Please enter your prompt. Press 'q' to quit.");
    if(userPrompt.ToLowerInvariant() == "q")
    {
        chatting = false;
        break;    
    }
    Console.WriteLine($"User: {userPrompt}");
    options.Messages.Add(new ChatRequestUserMessage(userPrompt));
    var assistantResponse = await client.GetChatCompletionsAsync(options);
    var response = assistantResponse.Value.Choices[0].Message.Content;
    Console.WriteLine($"Assistant: {response}");
    options.Messages.Add(new ChatRequestAssistantMessage(response));
};
options.Messages.Clear();

System: You are an AI assistant that helps people find information about your business
User: I want pizza
Assistant: Great! I can help you find a pizza place nearby. Could you please provide me with your location or a specific area?


## Prompt engineering in practice

In [11]:
var systemPrompt = 
    """
    You are an AI assistant that helps classify intent and provide customers with contact information to the appropriate department in your outdoor equipment business. 
    
    Only choose between the departments listed below. 
    
    If you don't know where to direct a customer, give them the customer service department. 
    
    You are friendly and greet customers with the message: "Hi, I am a customer service agent here to help direct you to the appropriate department"

    ## Context
    
    | Department | Number | Description |
    | Customer Service | 1-800-111-1110 | General questions |
    | Sales | 1-800-111-1111 | Purchase equipment and answer questions about products |
    | Billing | 1-800-111-1112 | Account and payment information |
    | Returns | 1-800-111-1113 | Return or exchange damaged equipment | 
    | Marketing | 1-800-111-1114 | Inquiries about business partnerships and marketing opportunities |

    ## Examples

    User: I want to buy a new set of hiking boots
    Assistant: Sure, here is the number for the sales department. 1-800-1111-1111

    User: My shipment arrived with damages. I'd like to see if I can exchange it.
    Assistant: Sorry to hear about your equipment being damaged. Here is the contact information for the returns department, 1-800-1111-1113

    User: I want to order a pizza.
    Assistant: Sorry, we are an outdoor equipment sales company and unfortunately don't sell pizza. 

    ## Negative Examples

    User: I'm interested in reselling some of your products in my shop. 
    Assistant: Sure, our billing department is able to help. Here is their number, 1-800-111-1113. 
    """;

In [12]:
options.Messages.Add(new ChatRequestSystemMessage(systemPrompt));

In [13]:
var chatting = true;

In [15]:
Console.WriteLine($"System: {systemPrompt}");
while(chatting)
{
    var userPrompt = await Kernel.GetInputAsync("Please enter your prompt. Press 'q' to quit.");
    if(userPrompt.ToLowerInvariant() == "q")
    {
        chatting = false;
        break;    
    }
    Console.WriteLine($"User: {userPrompt}");
    options.Messages.Add(new ChatRequestUserMessage(userPrompt));
    var assistantResponse = await client.GetChatCompletionsAsync(options);
    var response = assistantResponse.Value.Choices[0].Message.Content;
    Console.WriteLine($"Assistant: {response}");
    options.Messages.Add(new ChatRequestAssistantMessage(response));
};
options.Messages.Clear();

System: You are an AI assistant that helps classify intent and provide customers with contact information to the appropriate department in your outdoor equipment business. 

Only choose between the departments listed below. 

If you don't know where to direct a customer, give them the customer service department. 

You are friendly and greet customers with the message: "Hi, I am a customer service agent here to help direct you to the appropriate department"

## Context

| Department | Number | Description |
| Customer Service | 1-800-111-1110 | General questions |
| Sales | 1-800-111-1111 | Purchase equipment and answer questions about products |
| Billing | 1-800-111-1112 | Account and payment information |
| Returns | 1-800-111-1113 | Return or exchange damaged equipment | 
| Marketing | 1-800-111-1114 | Inquiries about business partnerships and marketing opportunities |

## Examples

User: I want to buy a new set of hiking boots
Assistant: Sure, here is the number for the sales depa